In [1]:
# hide
# default_exp utils.utils
import os
from nbdev.showdoc import *
if not os.path.exists('settings.ini'):
    os.chdir('..')

# Utils

In [2]:
# export
import sys
import os
import random as python_random
import logging
import shutil
from pathlib import Path
import re
import inspect 
import numpy as np

# block-types
import block_types.config.bt_defaults as dflt

## make_reproducible

In [3]:
# export
def make_reproducible ():
    """
    Make results obtained from neural network model reproducible. 
    
    This function should be run at the very beginning. The result 
    of calling this is that the pipeline produces the exact same 
    results as previous runs.
    """
    os.environ['CUDA_VISIBLE_DEVICES'] = ''
    os.environ['PYTHONHASHSEED'] = '0'

    # The below is necessary for starting Numpy generated random numbers
    # in a well-defined initial state.
    np.random.seed(123)

    # The below is necessary for starting core Python generated random numbers
    # in a well-defined state.
    python_random.seed(123)

    # The below set_seed() will make random number generation
    # in the TensorFlow backend have a well-defined initial state.
    # For further details, see:
    # https://www.tensorflow.org/api_docs/python/tf/random/set_seed
    try:
        import tensorflow as tf
        tf.random.set_seed(1234)
    except:
        print ('tensorflow needs to be installed in order to run make_reproducible()')

## set_logger

In [4]:
# export 
def set_logger (name, path_results='log', stdout=True,
                mode='w', just_message = False, filename='logs.txt',
                logging_level=logging.DEBUG, verbose=None):
    """Set logger."""
    logger = logging.getLogger(name)
    if verbose is not None:
        logging_level = logging.DEBUG if verbose == 2 else logging.INFO if verbose == 1 else logging.WARNING
    logger.setLevel(logging_level)

    for hdlr in logger.handlers[:]:  # remove all old handlers
        logger.removeHandler(hdlr)

    #if not logger.hasHandlers():

    # Create handlers
    if stdout:
        c_handler = logging.StreamHandler()
        c_handler.setLevel(logging_level)
        c_format = logging.Formatter('%(message)s')
        c_handler.setFormatter(c_format)
        logger.addHandler(c_handler)

    os.makedirs(path_results, exist_ok=True)
    f_handler = logging.FileHandler('%s/%s' %(path_results, filename), mode = mode)
    f_handler.setLevel(logging_level)
    if just_message:
        f_format = logging.Formatter('%(asctime)s - %(message)s')
    else:
        f_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s {%(filename)s:%(funcName)s:%(lineno)d} - %(message)s')
    f_handler.setFormatter(f_format)
    logger.addHandler(f_handler)
    logger.propagate = 0

    return logger

## remove_previous_results

In [5]:
# export
def remove_previous_results (path_results=dflt.path_results):
    """Remove folder containing previous results, if exists."""
    if Path(path_results).exists():
        shutil.rmtree(path_results)

## set_tf_loglevel

In [6]:
# export
def set_tf_loglevel(level):
    if level >= logging.FATAL:
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    if level >= logging.ERROR:
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
    if level >= logging.WARNING:
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
    else:
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
    logging.getLogger('tensorflow').setLevel(level)
    
    tf.get_logger().setLevel(level)

## store_attrs

In [7]:
#export
def argnames(f, frame=False):
    "Names of arguments to function or frame `f`"
    code = getattr(f, 'f_code' if frame else '__code__')
    return code.co_varnames[:code.co_argcount+code.co_kwonlyargcount]

In [8]:
#export
def _store_attr(self, overwrite=False, **attrs):
    stored = getattr(self, '__stored_args__', None)
    for n,v in attrs.items():
        if not overwrite and hasattr(self, n):
            continue
        setattr(self, n, v)
        if stored is not None: stored[n] = v

In [9]:
#export

def get_specific_dict_param (self, **kwargs):
    if (hasattr(self, 'name') and 
        kwargs.get(self.name) is not None and
        isinstance(kwargs[self.name], dict)):
        k = self.name
    elif (hasattr(self, 'class_name') and 
        kwargs.get(self.class_name) is not None and
        isinstance(kwargs[self.class_name], dict)):
        k = self.class_name
    elif (hasattr(self, 'hierarchy_level') and 
        kwargs.get(f'level_{self.hierarchy_level}') is not None and
        isinstance(kwargs[f'level_{self.hierarchy_level}'], dict)):
        k = f'level_{self.hierarchy_level}'
    elif (hasattr(self, 'group') and 
        kwargs.get(self.group) is not None and
        isinstance(kwargs[self.group], dict)):
        k = self.group
    else:
        k = None
    
    return k

def obtain_class_specific_attrs (self, **kwargs):
    """Overwrites parameters in kwargs with those found in a dictionary of the same name 
    given to this component.

    Checks if there is a parameter whose name is the name of the class or the name given 
    to this component. In that case, it overwrites the parameters in kwargs with those 
    found in that dictionary. The parameters in kwargs can be used as *global* parameters
    for multiple components, while parameters specific of one component can be set using 
    a dictionary with the name of that component. See example below.
    """
    k = get_specific_dict_param (self, **kwargs)
    
    if k is not None:
        config = kwargs[k]
    else:
        config = {}

    return config

In [168]:
# export
def get_hierarchy_level (base_class=object):
    stack = inspect.stack()
    hierarchy_level=0
    last_type = None
    for frame_number in range(1, len(stack)):
        fr = sys._getframe(frame_number)
        fr_stack = stack[frame_number]
        if fr is not fr_stack[0]:
            raise RuntimeError ('fr is not fr_stack[0]')
            
        args = argnames(fr, True)
        if len(args) > 0:
            self = fr.f_locals[args[0]]
            if last_type is None:
                last_type = type(self)
            if ((fr_stack.function == '__init__') and 
                isinstance(self, base_class) and 
                (type(self) != last_type) ):
                hierarchy_level += 1
                last_type = type(self)
    return hierarchy_level

In [178]:
def f (**kwargs):
    return B(**kwargs)

class A ():
    def __init__ (self, x=3, **kwargs):
        self.hierarchy_level = get_hierarchy_level(base_class=A)
        
class B(A):
    def __init__ (self, y=10, **kwargs):
        super().__init__ (**kwargs)
        self.ab = A (**kwargs)

class C(A):
    def __init__ (self, z=100, **kwargs):
        super().__init__ (**kwargs)
        self.a = A(**kwargs)
        self.b = f(**kwargs)
class D(A):
    def __init__ (self, h=100, **kwargs):
        super().__init__ (**kwargs)
        self.c = C(**kwargs)
        self.b = f(**kwargs)
a = A()
b = B()
c = C()
d = D()

assert (a.hierarchy_level==0 and b.hierarchy_level==0 and c.hierarchy_level==0
        and c.a.hierarchy_level==1 and c.b.hierarchy_level==1 and c.b.ab.hierarchy_level==2 
        and d.hierarchy_level == 0 and d.b.hierarchy_level==1 and d.b.ab.hierarchy_level==2
        and d.c.b.ab.hierarchy_level==3)

In [162]:
#export
def replace_attr_and_store (names=None, but='', store_args=None, 
                            recursive=True, base_class=object, 
                            replace_generic_attr=True, **attrs):
    """
    Replaces generic attributes and stores them into attrs in `self`.
        
    If kwargs contains an attribute called the same way as the class of
    self, all the keys in that dictionary are considered class-specific
    attributes whose value overwrites any attribute in kwargs of the same
    name.
    
    The function is called recursively in the hierarchy of parent classes, 
    from the leaf to the root class, until it reaches an ascendant that 
    is not an instance of `base_class`. 
    
    Most of the implementation is taken from fastcore library, `store_attrs`
    function.
    """
    frame_number=1
    stack = inspect.stack()
    original_type = None
    while True:
        fr = sys._getframe(frame_number)
        fr_stack = stack[frame_number]
        if fr is not fr_stack[0]:
            raise RuntimeError ('fr is not fr_stack[0]')
        
        args = argnames(fr, True)
        if recursive:
            if len(args) > 0:
                self = fr.f_locals[args[0]]
                if not isinstance(self, base_class):
                    break
                if fr_stack.function != '__init__':
                    break
                if original_type is None:
                    original_type = type(self)
                    
                if type(self) != original_type:
                    break
            else:
                break
        
        if store_args is None: store_args = not hasattr(self,'__slots__')
        if store_args and not hasattr(self, '__stored_args__'): self.__stored_args__ = {}
        if names and isinstance(names,str): names = re.split(', *', names)
        #pdb.set_trace()
        ns = names if names is not None else getattr(self, '__slots__', args[1:])
        added = {n:fr.f_locals[n] for n in ns}
        attrs = {**attrs, **added}
        if replace_generic_attr and 'kwargs' in fr.f_locals:
            class_specific_attrs = obtain_class_specific_attrs (self, **fr.f_locals['kwargs'])
            attrs.update(class_specific_attrs)
        if isinstance(but,str): but = re.split(', *', but)
        attrs = {k:v for k,v in attrs.items() if k not in but}
        _store_attr(self, **attrs)
        
        if not recursive:
            break
        
        frame_number += 1
        

In [163]:
import pytest

def f (**kwargs):
    return B(**kwargs)

class A ():
    def __init__ (self, x=3, **kwargs):
        replace_attr_and_store (base_class=A)
        
class B(A):
    def __init__ (self, y=10, **kwargs):
        super().__init__ (**kwargs)
        self.ab = A (**kwargs)

class C(A):
    def __init__ (self, z=100, **kwargs):
        super().__init__ (**kwargs)
        self.a = A(**kwargs)
        self.b = f(**kwargs)
a = A()
b = B()
c = C()

assert a.x==3 and b.y==10 and b.x==3 and c.z==100 and c.x==3 and c.a.x==3 and c.b.y==10 and c.b.x==3

with pytest.raises (AttributeError):
    print (c.y)

with pytest.raises (AttributeError):
    print (c.a.y)

with pytest.raises (AttributeError):
    print (c.a.z)

with pytest.raises (AttributeError):
    print (b.z)

with pytest.raises (AttributeError):
    print (b.ab.y)